In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
from utils.seed import fix_seed
import random
import os
from answer_judge.vqaEval import VQAEval

In [2]:
fix_seed(0)
data_reader=ReadData('/root/autodl-tmp/hallscope/VQA/train_qwen')
data=data_reader.read_all()
random.shuffle(data)

NameError: name 'fix_seed' is not defined

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,answers,question_id,most_likely,responses
0,[/root/autodl-fs/coco_images/train/COCO_train2...,[How many cows are in the image?\nAnswer the q...,"[{'answer': ['1'], 'answer_confidence': ['yes'...",[tensor(288117003)],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
1,[/root/autodl-fs/coco_images/train/COCO_train2...,[Who is flying this kite?\nAnswer the question...,"[{'answer': ['woman'], 'answer_confidence': ['...",[tensor(270636008)],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
2,[/root/autodl-fs/coco_images/train/COCO_train2...,[How many beds are here?\nAnswer the question ...,"[{'answer': ['3'], 'answer_confidence': ['mayb...",[tensor(133905002)],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
3,[/root/autodl-fs/coco_images/train/COCO_train2...,[What is the name of the establishment across ...,"[{'answer': ['presidio'], 'answer_confidence':...",[tensor(350077023)],{'embedding': [[[0.00579834 0.0213623 0.00665...,None
4,[/root/autodl-fs/coco_images/train/COCO_train2...,[Are there people on the bus?\nAnswer the ques...,"[{'answer': ['yes'], 'answer_confidence': ['ye...",[tensor(134620002)],{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None


In [5]:
df.shape

(11559, 6)

In [6]:
i=0

In [7]:
df.iloc[i]['img_path']

['/root/autodl-fs/coco_images/train/COCO_train2014_000000288117.jpg']

In [8]:
df.iloc[i]['most_likely']['response']

['1']

In [9]:
df.iloc[i]['question']

['How many cows are in the image?\nAnswer the question using a single word or phrase.\n']

In [10]:
df.iloc[i]['answers']

[{'answer': ['1'], 'answer_confidence': ['yes'], 'answer_id': tensor([1])},
 {'answer': ['1'], 'answer_confidence': ['yes'], 'answer_id': tensor([2])},
 {'answer': ['1'], 'answer_confidence': ['yes'], 'answer_id': tensor([3])},
 {'answer': ['1'], 'answer_confidence': ['yes'], 'answer_id': tensor([4])},
 {'answer': ['1'], 'answer_confidence': ['yes'], 'answer_id': tensor([5])},
 {'answer': ['1'], 'answer_confidence': ['yes'], 'answer_id': tensor([6])},
 {'answer': ['1'], 'answer_confidence': ['yes'], 'answer_id': tensor([7])},
 {'answer': ['1'], 'answer_confidence': ['yes'], 'answer_id': tensor([8])},
 {'answer': ['1'], 'answer_confidence': ['yes'], 'answer_id': tensor([9])},
 {'answer': ['1'], 'answer_confidence': ['yes'], 'answer_id': tensor([10])}]

In [11]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [12]:
embeddings=np.concatenate(embeddings,axis=0)

In [13]:
embeddings.shape

(11559, 29, 3584)

In [14]:
def transpose_lines(x):
    f=pd.DataFrame(x)
    transposed=f['answer'].apply(pd.Series).T.values.tolist()
    return transposed

df['answers']=df['answers'].apply(transpose_lines)

In [15]:
gt_label=df['answers'].to_list()
gt_label=np.concatenate(gt_label,axis=0).tolist()

In [16]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [17]:
rsps[0]

'1'

In [18]:
judge=VQAEval(gt_label,rsps)
judge.evaluate()
print(judge.evalQA)
print(judge.accuracy['overall'])

computing accuracy


  0%|          | 0/11559 [00:00<?, ?it/s]

100%|██████████| 11559/11559 [00:03<00:00, 3010.48it/s]

Done computing accuracy
{0: 100, 1: 100, 2: 100, 3: 100, 4: 100, 5: 100, 6: 100, 7: 100, 8: 0.0, 9: 100, 10: 100, 11: 100, 12: 100, 13: 66.67, 14: 100, 15: 100, 16: 100, 17: 66.67, 18: 100, 19: 100, 20: 100, 21: 0.0, 22: 66.67, 23: 100, 24: 100, 25: 100, 26: 100, 27: 100, 28: 0.0, 29: 0.0, 30: 100, 31: 100, 32: 100, 33: 100, 34: 100, 35: 100, 36: 100, 37: 100, 38: 100, 39: 100, 40: 100, 41: 100, 42: 0.0, 43: 100, 44: 100, 45: 100, 46: 100, 47: 100, 48: 0.0, 49: 100, 50: 100, 51: 100, 52: 100, 53: 100, 54: 100, 55: 100, 56: 0.0, 57: 100, 58: 100, 59: 100, 60: 100, 61: 100, 62: 100, 63: 100, 64: 100, 65: 100, 66: 100, 67: 100, 68: 100, 69: 33.33, 70: 100, 71: 100, 72: 100, 73: 100, 74: 100, 75: 100, 76: 100, 77: 100, 78: 100, 79: 100, 80: 100, 81: 100, 82: 100, 83: 100, 84: 100, 85: 100, 86: 100, 87: 100, 88: 100, 89: 100, 90: 66.67, 91: 100, 92: 100, 93: 100, 94: 100, 95: 100, 96: 100, 97: 100, 98: 100, 99: 100, 100: 100, 101: 100, 102: 100, 103: 100, 104: 0.0, 105: 66.67, 106: 100, 107

In [19]:
gt_scores=[judge.evalQA[i] for i in range(len(judge.evalQA))]

In [20]:
is_correct=[]
threshold=30
for i,x in enumerate(gt_scores):
    if x >=threshold:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [21]:
is_correct.shape

(11559,)

In [22]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 11559 responses. 10427 answer correctly and 1132 answer wrongly


In [23]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 2000
# exclude validation samples.
train_idxs = train_val_idxs[:- validset_len]  # trainset index
val_idxs = train_val_idxs[- validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:7247
validset length:2000
testset length:2312
trainset postive: 6518 negative: 729
valset postive: 1807 negative: 193
testset postive: 2102 negative: 210


In [24]:
all_embeddings = embeddings
all_embeddings.shape

(11559, 29, 3584)

In [25]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [33]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCALinear,PCAKernel,KernelPCA
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score
from ml_utils.metrics import auc_pr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [34]:
embedding_val.shape

(2000, 29, 3584)

In [37]:
M=3584
def evaluator(n_components,i_layer,X,y):
    discriminator=KernelPCA(X[:,i_layer,:],n_components,M,method='CoP',gamma=1)
    scores=discriminator.get_score(X[:,i_layer,:])
    
    return roc_auc_score(y,scores)
    #split=discriminator.get_best_split(scores,y)
    #y_pred=scores>split
    #return accuracy_score(y,y_pred)
    #return balanced_accuracy_score(y,y_pred)
    #
    #split=discriminator.get_best_split(y)
    #y_preds=(scores>split)
    #return accuracy_score(y,y_preds)

In [38]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,4),
    'layer':range(10,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

found best n_components:1,layer:10 Score: 0.7417125685661117
found best n_components:1,layer:11 Score: 0.7496007179907729
found best n_components:1,layer:12 Score: 0.7515562679390166
found best n_components:1,layer:13 Score: 0.7572250688886913
found best n_components:1,layer:14 Score: 0.7596135925058279
found best n_components:1,layer:17 Score: 0.7650960140616084
found best n_components:1,layer:18 Score: 0.7653196693342814
found best n_components:1,layer:19 Score: 0.7737239463112651
found best n_components:1,layer:20 Score: 0.7952005872384595


  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

In [29]:
best_paras

{'n_components': 3, 'layer': 21, 'best_score': np.float64(0.6251107523706025)}

In [39]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [41]:
discriminator=KernelPCA(embedding_val[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [42]:
discriminator=KernelPCA(embedding_train[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.8000551952532082 AUROC:0.7705084705812036


In [43]:
discriminator=KernelPCA(embedding_test[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_test[:,best_layer,:])
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.6647923875432526 AUROC:0.8039826016039148


Train Linear Prob to Get Better Results

In [44]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [45]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [46]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)} auroc:{roc_auc_score(y_test,scores_test)}  b-acc:{balanced_accuracy_score(y_test,pred_test)} auc-pr:{auc_pr(y_test,scores_test)}')

test set acc: 0.8140138408304498 f1:0.8913043478260869 auroc:0.793836935345023  b-acc:0.7026958452267682 auc-pr:0.9748968749988619
